# Schrödinger Equation from Symmetry Principles

This notebook contains the programmatic verification for the **Schrödinger Equation from Symmetry Principles** entry from the THEORIA dataset.

**Entry ID:** schrodingen_equation  
**Required Library:** sympy 1.12.0

## Description
The time-dependent Schrödinger equation governs the unitary time evolution of a nonrelativistic quantum state `psi(x,t)` under a Hamiltonian `H = p^2/(2m) + V(x,t)`. It specifies how probability amplitudes change in space and time, with measurement statistics given by ρ = |ψ|^2 (Born rule). This equation is the central dynamical law of nonrelativistic quantum mechanics and applies when velocities are small compared with c and spin/gauge effects are absent or handled by standard extensions.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.12.0

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# ===============================================
# Programmatic verification: Schrödinger (1D)
# Mirrors derivation Steps 2, 5–7, 8 with robust simplifications.
# Key choices to avoid false negatives:
#  - j uses antisymmetric form (no Im(...))
#  - V is declared real so conj(V)=V
# ===============================================

# Symbols & functions
x, t, hbar, m = sp.symbols('x t hbar m', positive=True, real=True)
psi = sp.Function('psi')(x,t)
V = sp.Function('V', real=True)(x,t)   # real scalar potential ⇒ conj(V)=V

# ---------- Step 5–6: TDSE ----------
tdse = sp.Eq(sp.I*hbar*sp.diff(psi, t), -hbar**2/(2*m)*sp.diff(psi, x, 2) + V*psi)
tdse_conj = sp.Eq(-sp.I*hbar*sp.diff(sp.conjugate(psi), t), -hbar**2/(2*m)*sp.diff(sp.conjugate(psi), x, 2) + V*sp.conjugate(psi))

# ---------- Step 7: Continuity equation ----------
rho = sp.conjugate(psi)*psi
j = (hbar/(2*sp.I*m))*(sp.conjugate(psi)*sp.diff(psi, x) - psi*sp.diff(sp.conjugate(psi), x))

# Replace time derivatives using TDSE and its conjugate
dpsi_t = sp.simplify((-sp.I/hbar)*(-hbar**2/(2*m)*sp.diff(psi, x, 2) + V*psi))
dpsic_t = sp.simplify(( sp.I/hbar)*(-hbar**2/(2*m)*sp.diff(sp.conjugate(psi), x, 2) + V*sp.conjugate(psi)))
rho_t = sp.diff(rho, t).subs({sp.diff(psi, t): dpsi_t, sp.diff(sp.conjugate(psi), t): dpsic_t})

# Residual of continuity equation: ∂t rho + ∂x j
residual = sp.simplify(sp.together(rho_t + sp.diff(j, x)))
assert sp.simplify(residual) == 0

# ---------- Step 8: WKB classical limit check (structure) ----------
R = sp.Function('R')(x,t)
S = sp.Function('S')(x,t)
psi_wkb = R*sp.exp(sp.I*S/hbar)
expr = sp.simplify(sp.I*hbar*sp.diff(psi_wkb,t) + hbar**2/(2*m)*sp.diff(psi_wkb,x,2) - V*psi_wkb)
expr = sp.simplify(expr*sp.exp(-sp.I*S/hbar))  # remove the global phase
Rx, Rxx = sp.diff(R,x), sp.diff(R,x,2)
St, Sx, Sxx = sp.diff(S,t), sp.diff(S,x), sp.diff(S,x,2)
HJ_quantum = St + Sx**2/(2*m) + V - (hbar**2/(2*m))*(Rxx/R)
Imag_part  = sp.diff(R,t) + (1/(2*m))*(2*Rx*Sx + R*Sxx)
recon = sp.simplify(R*HJ_quantum + sp.I*hbar*Imag_part)
assert sp.simplify(expr - recon) == 0

# Continuity from Imag part: 2R*Imag_part = ∂t(R^2) + (1/m)∂x(R^2 Sx)
cont_from_imag = sp.simplify(2*R*Imag_part - (sp.diff(R**2, t) + (1/m)*sp.diff(R**2*Sx, x)))
assert sp.simplify(cont_from_imag) == 0

print('Continuity and WKB checks passed ✔')


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=schrodingen_equation.json](https://theoria-dataset.org/entries.html?entry=schrodingen_equation.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0